In [113]:
# Vector Data impimentation
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [114]:
loader = PyPDFLoader("../test_documents/attention.pdf")

In [115]:
document = loader.load()

In [116]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents =  text_splitter.split_documents(document)
vectors = FAISS.from_documents(documents, embeddings)

In [117]:
retriver = vectors.as_retriever()

In [118]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, "Database_Search",
                      """Search for information about the research paper. You should use this tool to get the information about the relevant research 
                      paper.""")

In [119]:
Tools = [retriver_tool]

# Agent Implimentation 

In [526]:
## Agents
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain.tools import Tool
from langchain.llms import Ollama
from langchain import hub
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
load_dotenv()


True

In [515]:
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
parse_llm_with_tools = llm.bind_tools(Tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [516]:
parse_agent_prompt = ChatPromptTemplate.from_template("""You are the Research Document Parsing Agent. 
First Use the given Database_Search Tool to get research paper context and  You Read the given  research paper  Fully and divide them into Abstract, Method, Math, Experiments, Results.
Rules:
- Do NOT ask questions
- Do NOT wait for user input
- Use the document retrieval tool to read the paper
- Be concise but complete
{input}
{agent_scratchpad}
""")

In [517]:
parse_agent = create_tool_calling_agent(parse_llm_with_tools, Tools, parse_agent_prompt)
parse_agent_executor = AgentExecutor(agent=parse_agent, tools=Tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [569]:
result_doc_agent=parse_agent_executor.invoke({'input':"analyze the  given research paper"})



> Entering new AgentExecutor chain...

Invoking: `Database_Search` with `{'query': ''}`


tensorflow/tensor2tensor.
Acknowledgements We are grateful to Nal Kalchbrenner and Stephan Gouws for their fruitful
comments, corrections and inspiration.
9

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on atte

In [486]:
print(result_doc_agent['output'])

**Attention Is All You Need** – Structured Summary  
*(Ashish Vaswani et al., 2017)*  

---

### 1. Abstract  
The paper introduces the **Transformer**, a sequence‑to‑sequence model that replaces recurrent and convolutional layers with a stack of **self‑attention** mechanisms. The architecture is fully parallelizable, trains faster, and achieves state‑of‑the‑art BLEU scores on WMT 2014 English‑German (BLEU 28.4) and English‑French (BLEU 41.8) translation tasks.  

---

### 2. Method (Model Architecture)  

| Component | Description |
|-----------|-------------|
| **Encoder** | 6 identical layers. Each layer has: 1) *multi‑head self‑attention* (queries, keys, values all from encoder output), 2) *position‑wise feed‑forward network* (two linear layers with ReLU). Residual connections + layer‑norm around each sub‑layer. |
| **Decoder** | 6 identical layers. Each layer has: 1) *masked multi‑head self‑attention* (prevent future‑token access), 2) *encoder‑decoder attention* (queries from deco

# Teaching Agent

In [551]:
teaching_agent_prompt = ChatPromptTemplate.from_template("""You are the Teaching Assistant Agent. 
Read the given context carefully and explain the key concepts, Architecture ideas , algorithm intuition using 
simple language, Analogies and step by step reasoning so even a complete beginner could understand.Use the tools only if you want.
Compress the result into bullet points.
Keep only essential facts, findings, and conclusions.
Input: {input}
{agent_scratchpad}
""")

In [552]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper= wiki_api_wrapper)

In [553]:
llm = ChatGroq(
    model="openai/gpt-oss-safeguard-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
teach_tools = [wiki_tool]
teach_llm=llm.bind_tools(teach_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [554]:
teaching_agent = create_tool_calling_agent(teach_llm,teach_tools ,teaching_agent_prompt)
teaching_agent_executor = AgentExecutor(agent=teaching_agent, tools=teach_tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [246]:
teach_agent_results = teaching_agent_executor.invoke({"input":"Teach me this research paper context"+ "context - "+str(result_doc_agent['output'])})



> Entering new AgentExecutor chain...
## 🎓 “Attention Is All You Need” – A Beginner‑Friendly Guide

Imagine you’re translating a sentence from English to German.  
A traditional neural machine‑translation system would read the whole sentence word by word, keeping a running “memory” of what it has seen so far.  
The **Transformer** does something completely different: it looks at *all* words at once, decides *which* words matter for each other, and does this in a way that can be computed in parallel on a GPU.  
That’s why the paper’s title is so catchy – the whole model is built **only** on a mechanism called *attention*.

Below is a step‑by‑step walk‑through of the key ideas, with everyday analogies and simple math explanations.

---

### 1. The Big Picture: Encoder‑Decoder with Attention

| Component | What it does | Analogy |
|-----------|--------------|---------|
| **Encoder** | Reads the source sentence and turns it into a set of “context‑aware” vectors. | A group of people liste

In [247]:
print(teach_agent_results['output'])

## 🎓 “Attention Is All You Need” – A Beginner‑Friendly Guide

Imagine you’re translating a sentence from English to German.  
A traditional neural machine‑translation system would read the whole sentence word by word, keeping a running “memory” of what it has seen so far.  
The **Transformer** does something completely different: it looks at *all* words at once, decides *which* words matter for each other, and does this in a way that can be computed in parallel on a GPU.  
That’s why the paper’s title is so catchy – the whole model is built **only** on a mechanism called *attention*.

Below is a step‑by‑step walk‑through of the key ideas, with everyday analogies and simple math explanations.

---

### 1. The Big Picture: Encoder‑Decoder with Attention

| Component | What it does | Analogy |
|-----------|--------------|---------|
| **Encoder** | Reads the source sentence and turns it into a set of “context‑aware” vectors. | A group of people listening to a story and each writing down a 

# Mathematical Reasoning Agent

In [555]:
Math_agent_prompt = ChatPromptTemplate.from_template("""You are the Mathematical Reasoning Agent. 
Read the given research paper context carefully, extract all the mathematical formulas and explain , analyze equations, Loss functions, 
optimization methods.
Explain what each equation represents, why it needed and assumptions made so even a complete beginner could understand.
present the results in a human-readable format. 
Compress the result into bullet points.
Keep only essential facts, findings, and conclusions.
Use the provided tools ONLY if you need.
Input: {input}
{agent_scratchpad}

""")

In [556]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
math_tools = [wiki_tool]
math_llm=llm.bind_tools(math_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [557]:
math_agent = create_tool_calling_agent(math_llm,math_tools ,Math_agent_prompt)
math_agent_executor = AgentExecutor(agent=math_agent, tools=math_tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [538]:
math_agent_results = math_agent_executor.invoke({'input':'explain all the mathematical terms in this research paper'+ 'context- '+str(result_doc_agent['output'])})



> Entering new AgentExecutor chain...
## 📚  “Attention Is All You Need” – Mathematics Made Simple  

Below is a **step‑by‑step walk‑through** of every equation that appears in the paper, together with the *why* and the *assumptions* behind each term.  
The goal is to make the math understandable even if you have never seen a neural network before.

---

### 1️⃣  Notation that is used everywhere  

| Symbol | Meaning (in plain English) |
|--------|----------------------------|
| **\(x\)** | A vector (a list of numbers). |
| **\(X\in\mathbb{R}^{n\times d}\)** | A matrix with *n* rows and *d* columns; each row is a *d‑dimensional* vector. |
| **\(d_{\text{model}}\)** | Dimension of the model (the size of every token representation). Typical values: 512 (Base) or 1024 (Big). |
| **\(d_k, d_v\)** | Dimension of *keys* and *values* inside an attention head (usually \(d_k = d_v = d_{\text{model}}/h\)). |
| **\(h\)** | Number of attention heads (8 in the Base model, 16 in the Big model). |
|

In [276]:
print(math_agent_results['output'])

**Attention Is All You Need (Vaswani et al., 2017)**  
A beginner‑friendly walk‑through of every mathematical expression that appears in the paper, together with the intuition behind it, why it is needed, and the assumptions that underlie it.

---

## 1.  Quick Overview of the Model

| Part | What it does | Main symbols |
|------|--------------|--------------|
| **Input embedding** | Turns each token (word‑piece, character, etc.) into a dense vector of size `d_model`. | `x₁,…,xₙ ∈ ℝ^{d_model}` |
| **Positional encoding** | Gives the model a notion of “order” because attention itself is order‑agnostic. | `PE(pos, i)` |
| **Encoder** | 6 identical layers, each containing **(a)** Multi‑Head Self‑Attention and **(b)** a Position‑wise Feed‑Forward Network (FFN). | `Enc₁,…,Enc₆` |
| **Decoder** | 6 identical layers, each containing **(a)** Masked Multi‑Head Self‑Attention, **(b)** Multi‑Head Encoder‑Decoder Attention, **(c)** FFN. | `Dec₁,…,Dec₆` |
| **Output projection + softmax** | Turns t

# Experimental Analysis Agent

In [558]:
experimental_analysis_agent_prompt = ChatPromptTemplate.from_template("""You are the Experimental Analysis Agent and Empirical reviewer. 
Read the given context carefully and explain / Reviews Datasets used, Experimental setup, Baselines, Evaluation metrics.
Determines whether experiments support claims, If comparisons are fair. Conpare them with other related research papers.
Compress the result into bullet points.
Keep only essential facts, findings, and conclusions.
Use the given tools only if needed. Give me exacly a final answer don't think too much.
Input: {input}
{agent_scratchpad}
""")

In [559]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper= arxiv_wrapper)

In [560]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
experimental_analysis_agent_tools = [arxiv_tool,wiki_tool]
experimental_analysis_agent_llm=llm.bind_tools(experimental_analysis_agent_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [561]:
experimental_analysis_agent = create_tool_calling_agent(experimental_analysis_agent_llm,experimental_analysis_agent_tools ,experimental_analysis_agent_prompt)
experimental_analysis_agent_executor = AgentExecutor(agent=experimental_analysis_agent, tools=experimental_analysis_agent_tools, verbose=True,max_iterations=5,handle_parsing_errors=True)

In [394]:
experimental_results = experimental_analysis_agent_executor.invoke({'input':'experiment analysis on the given research paper '+'CONTEXT - '+str(result_doc_agent['output'])})



> Entering new AgentExecutor chain...
**Experimental‑Analysis & Empirical Review of “Attention Is All You Need” (Vaswani et al., 2017)**  

---

## 1. Datasets Used  

| Dataset | Language Pair | Size (≈ M sentences) | Pre‑processing | BPE merges | Test set |
|---------|---------------|----------------------|----------------|-----------|----------|
| **WMT 2014** | English → German (EN‑DE) | 4.5 M | Tokenized, lower‑cased, SentencePiece (or BPE) | 32 k | *newstest2014* |
| **WMT 2014** | English → French (EN‑FR) | 36 M | Same pipeline as EN‑DE | 32 k | *newstest2014* |

*Why these datasets?* They are the standard benchmarks for neural machine translation (NMT) and were used by all prior strong baselines (GNMT, ConvS2S, LSTM‑attention). This makes direct BLEU comparison possible.

---

## 2. Experimental Setup  

| Component | Configuration |
|-----------|----------------|
| **Model variants** | *Transformer‑base* (6 encoder + 6 decoder layers, d_model = 512, h = 8, d_ff = 2048) and *

In [395]:
print(experimental_results['output'])

**Experimental‑Analysis & Empirical Review of “Attention Is All You Need” (Vaswani et al., 2017)**  

---

## 1. Datasets Used  

| Dataset | Language Pair | Size (≈ M sentences) | Pre‑processing | BPE merges | Test set |
|---------|---------------|----------------------|----------------|-----------|----------|
| **WMT 2014** | English → German (EN‑DE) | 4.5 M | Tokenized, lower‑cased, SentencePiece (or BPE) | 32 k | *newstest2014* |
| **WMT 2014** | English → French (EN‑FR) | 36 M | Same pipeline as EN‑DE | 32 k | *newstest2014* |

*Why these datasets?* They are the standard benchmarks for neural machine translation (NMT) and were used by all prior strong baselines (GNMT, ConvS2S, LSTM‑attention). This makes direct BLEU comparison possible.

---

## 2. Experimental Setup  

| Component | Configuration |
|-----------|----------------|
| **Model variants** | *Transformer‑base* (6 encoder + 6 decoder layers, d_model = 512, h = 8, d_ff = 2048) and *Transformer‑big* (d_model = 1024, h = 16

# Testing Multi Agent Architecture

In [170]:
from langchain.tools import tool

In [ ]:
You are the Teaching Assistant Agent. 
Read the given context carefully and explain the key concepts, Architecture ideas , algorithm intuition using 
simple language, Analogies and step by step reasoning so even a complete beginner could understand.Use the tools only if you want.
Input: {input}
{agent_scratchpad}
<context>
{context}
</context>

In [562]:
@tool
async def teach_paper_tool(request: str, context: str) -> str:
    """Teach the given research paper context in a beginner friendly manner.

    Use this tool to explain the key concepts, Architecture ideas , algorithm intuition using 
    simple language, Analogies and step by step reasoning so even a complete beginner could understand.
    Compress the output content into bullet points.
    Keep only essential facts, findings, and conclusions.

    """
    combined_input = f"""
TASK:
{request}

RESEARCH PAPER CONTEXT:
{context}
"""
    result = await teaching_agent_executor.ainvoke({
        "input": combined_input
    })

    return result['output']




In [563]:
@tool
async def teach_math_tool(request: str, context: str) -> str:
    """Read the given research paper context carefully, extract all the mathematical formulas and explain , analyze equations, Loss functions, 
    optimization methods.

    Use this tool to Explain what each equation represents, why it needed and assumptions made so even a complete beginner could understand.
    present the results in a human-readable format.

    """
    combined_input = f"""
    TASK:
    {request}
    
    RESEARCH PAPER CONTEXT:
    {context}
    """
    result = await math_agent_executor.ainvoke({
        "input": combined_input
    })

    return compressed_context['output']

In [564]:
@tool
async def experiment_analysis_tool(request: str, context: str) -> str:
    """
    Use this tool to conduct the Experimental Analysis
    Read the given context carefully and explain / Reviews Datasets used, Experimental setup, Baselines, Evaluation metrics.
    Determines whether experiments support claims, If comparisons are fair. Conpare them with other related research papers.
    Only use a sufficient token amount so supervisor agent can handle it
    

    """
    combined_input = f"""
    TASK:
    {request}
    
    RESEARCH PAPER CONTEXT:
    {context}
    """
    result = await experimental_analysis_agent_executor.ainvoke({
        "input": combined_input
    })

    return compressed_context['output']

# Summarizer Agent

In [548]:
summarizer_agent_prompt = ChatPromptTemplate.from_template("""
    Compress the following content into bullet points.
    Keep only essential facts, findings, and conclusions.
    
    Content:
    {input}
    {agent_scratchpad}
    """) 

summary_llm = ChatGroq(
    model="openai/gpt-oss-safeguard-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params... 
)


#supervisor_tools = [teach_paper_tool, teach_math_tool, experiment_analysis_tool]
#supervisor_llm_with_tools =llm_supervisor.bind_tools(supervisor_tools)
summary_agent = create_tool_calling_agent(summary_llm,[wiki_tool], summarizer_agent_prompt)
summary_agent_executor = AgentExecutor(agent=summary_agent, tools=[wiki_tool], verbose=True,max_iterations=2,handle_parsing_errors=True)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [549]:
summarized_context = summary_agent_executor.invoke({"input":result_doc_agent["output"]})



> Entering new AgentExecutor chain...
**Attention Is All You Need – Key Take‑aways**

- **Model Innovation**
  - Introduces the **Transformer**, a sequence‑to‑sequence architecture that uses only **self‑attention** (no RNNs or CNNs).
  - Fully parallelizable, enabling faster training and inference.

- **Architecture Highlights**
  - **Encoder**: 6 layers × (multi‑head self‑attention + position‑wise feed‑forward) with residual + layer‑norm.
  - **Decoder**: 6 layers × (masked self‑attention + encoder‑decoder attention + feed‑forward) with residual + layer‑norm.
  - **Multi‑head Attention**: 8 heads (Base) or 16 heads (Big); each head performs scaled dot‑product attention.
  - **Positional Encoding**: Sinusoidal signals added to embeddings; essential for sequence order (drop ~1.5 BLEU if removed).

- **Mathematical Core**
  - Scaled dot‑product: `Attention(Q,K,V)=softmax(QKᵀ/√d_k)V`.
  - Multi‑head: concatenate heads → linear projection.
  - Feed‑forward: two linear layers with ReLU.
 

In [542]:
print(summarized_context['output'])

**Key Take‑aways from “Attention Is All You Need”**

- **Model Innovation**  
  - Introduces the **Transformer**: a sequence‑to‑sequence architecture that uses only self‑attention, no recurrence or convolutions.  
  - Fully parallelizable, enabling faster training and inference.

- **Architecture Highlights**  
  - **Encoder**: 6 layers × (multi‑head self‑attention + position‑wise feed‑forward).  
  - **Decoder**: 6 layers × (masked self‑attention + encoder‑decoder attention + feed‑forward).  
  - **Multi‑head attention**: 8 heads (base) or 16 heads (big); each head performs scaled dot‑product attention.  
  - **Positional encoding**: sinusoidal signals added to token embeddings; essential for sequence order.  
  - Residual connections + layer‑norm around every sub‑layer; final linear + softmax for output.

- **Training Regimen**  
  - Optimizer: Adam (β₁=0.9, β₂=0.98, ε=1e‑9).  
  - Learning‑rate schedule: warm‑up 4000 steps → decay ∝ step⁻¹/².  
  - Dropout 0.1, label smoothing ε=0.1

In [565]:
SUPERVISOR_PROMPT = ChatPromptTemplate.from_template(
    """You are the research coordinator agent. 
    Read the given research paper context
    You can Breaks the paper into logical components such as problem, Method, Math (use the teach_math_tool for this),   
    Theory (use the teach_paper_tool for this), Experiments (use the experiment_analysis_tool for this), Results
    Assigns tasks to specialist agents, Controls analysis order, Provide agent inputs only with the relavant contenxt using the given tool.
    Give ONLY relevent CONTEXT TO EACH AGENT / TOOLS WITH INPUT.
    teach_math_tool - to explain math
    teach_paper_tool - to explain the paper content
    Decides when analysis is “complete” 
    Break down user requests into appropriate tool calls and coordinate the results. 
    When a request involves multiple actions, use multiple tools in sequence.
    finally you should give a full blown analysis report. Not Just A Summary

    Input: {input}
    {agent_scratchpad}
    <context>
    {context}
    </context>"""
)

In [566]:

llm_supervisor = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)
supervisor_tools = [teach_paper_tool, teach_math_tool, experiment_analysis_tool]
supervisor_llm_with_tools =llm_supervisor.bind_tools(supervisor_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [567]:
supervisor_agent = create_tool_calling_agent(supervisor_llm_with_tools,supervisor_tools, SUPERVISOR_PROMPT)
supervisor_agent_executor = AgentExecutor(agent=supervisor_agent, tools=supervisor_tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [570]:
import asyncio

async def run_pipeline():

    result_doc_agent= await parse_agent_executor.ainvoke({'input':"analyze the  given research paper"})


    summary_context= await summary_agent_executor.ainvoke({"input":result_doc_agent["output"]})
    
    result = await supervisor_agent_executor.ainvoke({
        "input": f"""
Analyze and explain the following research paper.
Use provided tools when appropriate.

"""
    ,'context':summary_context['output']})

    return summary_context["output"]


# Run
final_output = await run_pipeline()
print(final_output)




> Entering new AgentExecutor chain...
I’m sorry, but no research paper was provided for analysis.

> Finished chain.


> Entering new AgentExecutor chain...
- No research paper was provided for analysis.

> Finished chain.


> Entering new AgentExecutor chain...
I’m sorry, but there’s no research‑paper content in the request for me to analyze. Please provide the text (or a link) of the paper you’d like examined, and I’ll break it down, run the appropriate tool calls, and produce a full analysis report.

> Finished chain.
- No research paper was provided for analysis.


In [507]:
print(final_output)

**Attention Is All You Need – Key Take‑aways**

- **Core Idea**
  - Introduces the **Transformer**: a seq‑to‑seq model that uses **only attention mechanisms** (no RNNs or convolutions).
  - Claims higher translation quality, greater parallelism, and faster training/inference.

- **Architecture**
  - **Encoder**: 6 identical layers → multi‑head self‑attention + position‑wise feed‑forward network.
  - **Decoder**: 6 identical layers → masked multi‑head self‑attention, encoder‑decoder attention, feed‑forward network.
  - **Attention**: Scaled dot‑product \( \text{softmax}(QK^\top/\sqrt{d_k})V \).
  - **Multi‑head**: splits model dimension into \(h\) heads, processes in parallel, then concatenates.
  - **Positional Encoding**: sinusoidal functions inject sequence order information.
  - **Embeddings**: learned token embeddings; final linear + softmax for output distribution.

- **Mathematical Formulas**
  - Scaled dot‑product attention (Eq. 1).
  - Multi‑head attention (Eq. 2): \( \text{Con